In [1]:
import sktime
import pandas as pd
import numpy as np

def readcsv(csv_loc:str, name:str):
    csv = pd.read_csv(csv_loc)

    # print(csv.to_string())
    metrics = csv["identifier"].to_list()
    timestamps = csv.columns[1:].to_flat_index()
    timestamps = timestamps.to_numpy().tolist()
    timestamps = pd.Int64Index(timestamps, dtype=np.int64)
    print(timestamps)
    index = pd.MultiIndex.from_product([[name], timestamps], names=['instances','timepoints'])
    print(index)
    vals = csv.drop(labels="identifier",axis=1).to_numpy().transpose()
    s = pd.DataFrame(vals, index=index, columns=metrics)
    return s



simple = readcsv("../Test_datasets/simple1.csv", "simple1")
simple2 = readcsv("../Test_datasets/simple2.csv", "simple1")
from sktime.transformations.panel.rocket import Rocket

rocket = Rocket()
rocket.fit(simple)
rocket.fit(simple2)
simple


C:\Users\nordb\AppData\Local\Temp\ipykernel_14852\2314209265.py:12: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  timestamps = pd.Int64Index(timestamps, dtype=np.int64)
C:\Users\nordb\AppData\Local\Temp\ipykernel_14852\2314209265.py:12: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  timestamps = pd.Int64Index(timestamps, dtype=np.int64)


Int64Index([1, 2, 3, 4, 5, 6, 7, 8], dtype='int64')
MultiIndex([('simple1', 1),
            ('simple1', 2),
            ('simple1', 3),
            ('simple1', 4),
            ('simple1', 5),
            ('simple1', 6),
            ('simple1', 7),
            ('simple1', 8)],
           names=['instances', 'timepoints'])
Int64Index([1, 2, 3, 4, 5, 6, 7, 8], dtype='int64')
MultiIndex([('simple1', 1),
            ('simple1', 2),
            ('simple1', 3),
            ('simple1', 4),
            ('simple1', 5),
            ('simple1', 6),
            ('simple1', 7),
            ('simple1', 8)],
           names=['instances', 'timepoints'])


first  second  third
instances timepoints                      
simple1   1             312       0     45
          2             345       0     87
          3             355       0     23
          4             355       0     67
          5             356       1     12
          6             354       1      3
          7             355       1    654
          8             356       2      4

## Below is my some code I will use for formatting unix timestamps to datetimeIndex, not relevant  ##

In [2]:
#import sktime
import pandas as pd
import numpy as np

def readcsv(csv_loc:str):
    csv = pd.read_csv(csv_loc).transpose()
    metrics = csv.loc["identifier"].to_list()
    timestamps = pd.DatetimeIndex(np.array(csv.index[1:].astype('int').tolist(), dtype='datetime64[s]'))
    print(csv.values[1:].flatten())
    vals = csv.values[1:].flatten()
    index = pd.MultiIndex.from_product([metrics, timestamps], names=['instances','timepoints'])
    s = pd.DataFrame(vals, index=index)
    return s
    # metrics = csv["identifier"].to_list()
    # timestamps = csv.columns[1:].to_flat_index()
    # timestamps = timestamps.to_numpy().tolist()
    # timestamps = pd.Int64Index(timestamps, dtype=np.int64)
    
    # index = pd.MultiIndex.from_product([metrics, timestamps], names=['instances','timepoints'])
    # vals = csv.drop(labels="identifier",axis=1).to_numpy().flatten()
    # s = pd.DataFrame(vals, index=index)
    # return s



simple = readcsv("../DTW/testing.csv")
simple



[0 10 13 0.1 9.0 75.0 0.2 8.0 1.0 0.22 7.0 0.5 0.3 6.0 24.0 0.3 5.0 7.0
 0.3 4.0 5.0 0.4 3.0 23.0 0.5 3.0 56.0]


0
instances timepoints               
increase  2023-01-18 12:32:47     0
          2023-01-18 12:32:52    10
          2023-01-18 12:32:57    13
          2023-01-18 12:33:02   0.1
          2023-01-18 12:33:07   9.0
          2023-01-18 12:33:12  75.0
          2023-01-18 12:33:17   0.2
          2023-01-18 12:33:22   8.0
          2023-01-18 12:33:27   1.0
decrease  2023-01-18 12:32:47  0.22
          2023-01-18 12:32:52   7.0
          2023-01-18 12:32:57   0.5
          2023-01-18 12:33:02   0.3
          2023-01-18 12:33:07   6.0
          2023-01-18 12:33:12  24.0
          2023-01-18 12:33:17   0.3
          2023-01-18 12:33:22   5.0
          2023-01-18 12:33:27   7.0
neither   2023-01-18 12:32:47   0.3
          2023-01-18 12:32:52   4.0
          2023-01-18 12:32:57   5.0
          2023-01-18 12:33:02   0.4
          2023-01-18 12:33:07   3.0
          2023-01-18 12:33:12  23.0
          2023-01-18 12:33:17   0.5
          2023-01-18 12:33:22   3.0
          2023-01-18 12:33:27  56.0

In [3]:
import pandas as pd
import numpy as np

def merge_with_new(existing_vals:pd.DataFrame, new_vals:pd.DataFrame) -> np.ndarray :
    common_columns = existing_vals.columns.intersection(new_vals.columns)
    reduced_existing = existing_vals.reindex(columns=common_columns)
    reduced_new = new_vals.reindex(columns=common_columns)
    timestamps = reduced_existing.index.get_level_values(1)

    row = existing_vals.iloc[(slice(None),0)]
    print(row.index)
    
    newnames = reduced_new.index.get_level_values(0).drop_duplicates()
    oldnames = reduced_existing.index.get_level_values(0).drop_duplicates()
    combined_names = oldnames.union(newnames)

    tuples = [(i,j) for i in combined_names for j in timestamps]
    index = pd.MultiIndex.from_tuples(tuples)

    reduced_combined = pd.concat([reduced_existing,reduced_new],axis=0)
    reduced_combined = reduced_combined.set_index(index)
    #remove any columns that exists in one but not the other
    return reduced_combined

def make_datetime_index(timestamps:list) -> pd.DatetimeIndex:
    beginning = timestamps[0]
    end = timestamps[-1]
    beginning, end = pd.to_datetime((beginning,end), unit="s")
    index = pd.date_range(start=beginning, end=end, periods=len(timestamps))
    return index

def concat_dfs(old:pd.DataFrame, new:pd.DataFrame) -> pd.DataFrame:
    unioned = old.index.union(new.index)
    concated = pd.concat([old.reindex(unioned), new.reindex(unioned)])
    return concated

def readcsv(csv_loc:str, num:int):
    csv = pd.read_csv(csv_loc)
    metrics = csv["identifier"].to_list()
    timestamps = csv.columns[1:].to_flat_index()
    timestamps = timestamps.to_numpy().tolist()
    timestamps = make_datetime_index(timestamps)
    index = pd.MultiIndex.from_product([[num], timestamps], names=['instances','timepoints'])
    vals = csv.drop(labels="identifier",axis=1).to_numpy().transpose()
    s = pd.DataFrame(vals, index=index, columns=metrics)
    return s

def readcsv_modified(csv_loc:str):
    csv = pd.read_csv(csv_loc)
    metrics = csv["identifier"].to_list()
    timestamps = csv.columns[1:].to_flat_index()
    timestamps = timestamps.to_numpy().tolist()
    timestamps = make_datetime_index(timestamps)
   # index = pd.MultiIndex.from_product([[num], timestamps], names=['instances','timepoints'])
    vals = csv.drop(labels="identifier",axis=1).to_numpy().transpose()
    s = pd.DataFrame(vals, index=timestamps, columns=metrics)
    return s

def removeNaNs(df:pd.DataFrame) -> pd.DataFrame:
    for col in df.columns:
        if df[col].isna().any():
            df = df.drop(col, axis=1)
    return df

def removeUniqueColumns(first:pd.DataFrame, second:pd.DataFrame) -> tuple:
    common_columns = first.columns.intersection(second.columns)
    reduced_old = first.reindex(columns=common_columns)
    reduced_new = second.reindex(columns=common_columns)
    return reduced_old, reduced_new

def removeUniqueColumns2(frames:list) -> list:
    template:pd.DataFrame = frames[0]
    #Round 1: Reduces the first frame to be the smallest size to match with everyone
    for frame in frames[1:]:
        common_columns = template.columns.intersection(frame.columns)
        template = template.reindex(columns=common_columns)
    returnlist = [template]
    #Round 2: Reduces all the other ones to the same minimum
    for frame in frames[1:]:
        common_columns = template.columns.intersection(frame.columns)
        returnlist.append(frame.reindex(columns=common_columns))
    return returnlist

def remove_monotonically_increasing_rows(df_list:list):
    x:pd.DataFrame
    returnlist = []
    for x in df_list:
        dropme = []
        for column in x.columns:
            if x[column].is_monotonic_increasing or x[column].is_monotonic_decreasing:
                dropme.append(column)
        returnlist.append(x.drop(dropme,axis=1))
    return returnlist

def remove_monotonic_rows(df_list:list):
    x:pd.DataFrame
    returnlist = []
    for x in df_list:
        row_count = x.apply(pd.Series.nunique, axis=1)
        returnlist.append(x[row_count > 1])
    return returnlist


def readcsvs(csv_loc_list:list):
    individual_dataframes = []
    for i in range(len(csv_loc_list)):
        individual_dataframes.append(readcsv_modified(csv_loc_list[i])) #time series

    #monotonic rows create issues in the data by adding noise that doesn't help
    #Instead of all this, maybe just do dtw
    #individual_dataframes = remove_monotonic_rows(individual_dataframes)
    individual_dataframes = remove_monotonically_increasing_rows(individual_dataframes)
    #Here: Go through the loop once again, start trimming. compare everything to element at 0, trim with it so it stays as the leanest version.

    removed_nans = []
    for frame in individual_dataframes:
        removed_nans.append(removeNaNs(frame))

    # initial_df = removed_nans[0]
    # removed_unique_cols = []
    # for frame in removed_nans[1:]:
    #     reduced_frames = removeUniqueColumns(initial_df, frame)
    #     initial_df = reduced_frames[0]
    #     removed_unique_cols.append(reduced_frames[1])
    
    # removed_unique_cols.append(initial_df)
    removed_unique_cols = removeUniqueColumns2(removed_nans)
    concated = pd.concat(removed_unique_cols, keys=[f'csv {i}' for i in range(1, len(removed_unique_cols)+1)])

    #concated.index = pd.MultiIndex.from_tuples([(idx, date) for idx, date in zip(concated.index.get_level_values(0), concated.index.get_level_values(1))], names=['files','times'])
    #concated2 = concated.reindex(index=pd.MultiIndex.from_tuples([(idx, date) for idx, date in zip(concated.index.get_level_values(0), concated.index.get_level_values(1))], names=['files','times']))
    #print(concated2)
    #return removed_unique_cols
    return concated


def present_data(list_of_ndarrays:list) -> pd.DataFrame:
    return pd.concat(list_of_ndarrays, keys=[f'csv {i}' for i in range(1, len(list_of_ndarrays)+1)])

def add_to_frame(csv_loc:str, frame:pd.DataFrame, name:str):
    #call readcsv to get a finished dataframe with the things in it. 
    #get an index of potentially different length that has to be cleaned.
    current_highest_num = frame.index.get_level_values('instances').max() 
    newframe = readcsv(csv_loc, name, current_highest_num)

    return concat_dfs(frame, newframe)
    csv = pd.read_csv(csv_loc)
    new_vals = csv.drop(labels="identifier", axis=1).to_numpy().transpose()
    #vals = np.append(frame.values, new_vals)
    vals = np.array(frame.values)
    merged = merge_with_new(vals, new_vals)
    added = np.concatenate([vals, new_vals],axis=0)
    print(added, added.shape)
    #Get a copy of the date index, expand it with another name
    index = frame.index
    name_index = index.levels[0].to_list()
    name_index.append(name)
    timepoints = index.get_level_values('timepoints')
    new_index = pd.MultiIndex.from_product([name_index, timepoints], names=['instances','timepoints'])
    

    df = pd.DataFrame(vals,  index=new_index, columns=frame.columns)
    return df

def make_list_of_vals(input:pd.DataFrame) -> list:
    newlist = []
    x:str
    for x in input.index.get_level_values(0).unique():
        newlist.append(input.loc[x].values)
    return newlist

def make_list_of_dfs(input:pd.DataFrame) -> list:
    newlist = []
    x:str
    for x in input.index.get_level_values(0).unique():
        newlist.append(input.loc[x])
    return newlist

def get_dimensions(input:list):
    return np.array(input).shape


from get_all_metrics_with_tags import get_all_metrics_with_tags
better_file_list, y = get_all_metrics_with_tags(r"F:\Master\Kubernetes\sockshop\microservices-demo\query\automated\generated_csvs_2")
print(better_file_list, y)
#print(complete_df)

#fitted = scaler.fit_transform(complete_df)
#rocket.fit(fitted,y)
#new_df = pd.DataFrame(data=fitted, index = complete_df.index, columns=complete_df.columns)
X = readcsvs(better_file_list)
X

#Next: reshape and re-reshape the matrices to allow or scaling. Why did it work before?


['F:/Master/Kubernetes/sockshop/microservices-demo/query/automated/generated_csvs_2/carts/rand 1678978299.0_663.csv', 'F:/Master/Kubernetes/sockshop/microservices-demo/query/automated/generated_csvs_2/carts/rand 1678981388.0_1546.csv', 'F:/Master/Kubernetes/sockshop/microservices-demo/query/automated/generated_csvs_2/carts/rand 1678984479.0_1202.csv', 'F:/Master/Kubernetes/sockshop/microservices-demo/query/automated/generated_csvs_2/carts/rand 1678987567.0_917.csv', 'F:/Master/Kubernetes/sockshop/microservices-demo/query/automated/generated_csvs_2/carts/rand 1678990654.0_642.csv', 'F:/Master/Kubernetes/sockshop/microservices-demo/query/automated/generated_csvs_2/carts/rand 1678993741.0_1403.csv', 'F:/Master/Kubernetes/sockshop/microservices-demo/query/automated/generated_csvs_2/catalog/rand 1678978299.0_996.csv', 'F:/Master/Kubernetes/sockshop/microservices-demo/query/automated/generated_csvs_2/catalog/rand 1678981388.0_1211.csv', 'F:/Master/Kubernetes/sockshop/microservices-demo/query

go_gc_duration_seconds&catalogue:80&catalogue&0.25  \
csv 1  2023-03-16 14:40:39                                           0.000367    
       2023-03-16 14:40:44                                           0.000367    
       2023-03-16 14:40:49                                           0.000367    
       2023-03-16 14:40:54                                           0.000366    
       2023-03-16 14:40:59                                           0.000366    
...                                                                       ...    
csv 31 2023-03-16 19:08:41                                           0.000347    
       2023-03-16 19:08:46                                           0.000347    
       2023-03-16 19:08:51                                           0.000347    
       2023-03-16 19:08:56                                           0.000347    
       2023-03-16 19:09:01                                           0.000347    

                            go_gc_duration_seconds&catalogue:80&catalogue&0.5  \
csv 1  2023-03-16 14:40:39                                           0.000444   
       2023-03-16 14:40:44                                           0.000444   
       2023-03-16 14:40:49                                           0.000444   
       2023-03-16 14:40:54                                           0.000441   
       2023-03-16 14:40:59                                           0.000441   
...                                                                       ...   
csv 31 2023-03-16 19:08:41                                           0.000366   
       2023-03-16 19:08:46                                           0.000366   
       2023-03-16 19:08:51                                           0.000366   
       2023-03-16 19:08:56                                           0.000366   
       2023-03-16 19:09:01                                           0.000366   

                            go_gc_duration_seconds&payment:80&payment&0.25  \
csv 1  2023-03-16 14:40:39                                        0.000329   
       2023-03-16 14:40:44                                        0.000328   
       2023-03-16 14:40:49                                        0.000328   
       2023-03-16 14:40:54                                        0.000328   
       2023-03-16 14:40:59                                        0.000327   
...                                                                    ...   
csv 31 2023-03-16 19:08:41                                        0.000332   
       2023-03-16 19:08:46                                        0.000332   
       2023-03-16 19:08:51                                        0.000332   
       2023-03-16 19:08:56                                        0.000332   
       2023-03-16 19:09:01                                        0.000332   

                            go_gc_duration_seconds&payment:80&payment&0.5  \
csv 1  2023-03-16 14:40:39                                       0.000362   
       2023-03-16 14:40:44                                       0.000362   
       2023-03-16 14:40:49                                       0.000362   
       2023-03-16 14:40:54                                       0.000362   
       2023-03-16 14:40:59                                       0.000360   
...                                                                   ...   
csv 31 2023-03-16 19:08:41                                       0.000348   
       2023-03-16 19:08:46                                       0.000348   
       2023-03-16 19:08:51                                       0.000348   
       2023-03-16 19:08:56                                       0.000348   
       2023-03-16 19:09:01                                       0.000348   

                            go_gc_duration_seconds&payment:80&payment&0.75  \
csv 1  2023-03-16 14:40:39                                        0.000445   
       2023-03-16 14:40:44                          

In [4]:
#######PREPROCESSING#############
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sktime.transformations.panel.rocket import Rocket
from sktime.transformations.panel.rocket import MiniRocketMultivariate
from sklearn.preprocessing import normalize
from sktime.datatypes import check_raise
from sktime.classification.kernel_based import RocketClassifier
from sklearn.linear_model import RidgeClassifierCV
from sktime.datatypes import convert_to


converted = convert_to(X, to_type=['pd-multiindex','numpyflat'])

vals_list = make_list_of_vals(X)
df_list = make_list_of_dfs(X)

rocket = Rocket()
minirocket = MiniRocketMultivariate()
rocketclassifier = RocketClassifier()
ridgeclassifier = RidgeClassifierCV(alphas=np.logspace(-3, 3, 10))
scaler = StandardScaler(with_mean=False)

def standard_scale(train, test):
    scaler = StandardScaler()
    train_scaled = []
    test_scaled = []

    for df_train in train:
        train_scaled.append(pd.DataFrame(data = scaler.fit_transform(df_train.values), index=df_train.index))
    
    for df_test in test:
        test_scaled.append(pd.DataFrame(data=scaler.fit_transform(df_test.values), index=df_test.index))

    return train_scaled, test_scaled

def minmax_scale(train, test):
    minmax = MinMaxScaler()

#scaled = scaler.fit_transform(X)
X_train,X_test, y_train, y_test = train_test_split(df_list, y)
rocketclassifier.fit(X_train, np.array(y_train))
print(rocketclassifier.score(X_train,y_train)) 


minirocket.fit(X_train)
X_train_transform = minirocket.transform(X_train)
X_train_scaled_transform = scaler.fit_transform(X_train_transform)

classifier = RidgeClassifierCV(alphas=np.logspace(-3,3,10))
classifier.fit(X_train_scaled_transform, y_train)
X_test_transform = minirocket.transform(X_test)
X_test_scaled_transform = scaler.transform(X_test_transform)
classifier.score(X_test_scaled_transform, y_test)




0.2608695652173913


0.0